In [1]:
import pandas as pd
import pickle
from sklearn.pipeline import FeatureUnion
from capstone_project import preprocessor

In [2]:
#the current working directory for python is the capstone_project/notebook folder
file_directory = "../output/data/"

train_data = pd.read_pickle(file_directory+"train_data.pkl")
train_data.fillna("", inplace=True)

test_data = pd.read_pickle(file_directory+"test_data.pkl")
test_data.fillna("", inplace=True)

In [3]:
train_data = train_data[:30]
test_data = test_data[:30]

tfidf = preprocessor.TfidfTransformer()
feature_egineering = preprocessor.FeatureTransformer()

train_y = train_data["is_duplicate"].values
test_y = test_data["is_duplicate"].values

feature_creator = FeatureUnion([('feature_engineering', feature_egineering), ('tfidf', tfidf)])

feature_creator.fit(train_data, train_y)

FeatureUnion(n_jobs=1,
       transformer_list=[('feature_engineering', FeatureTransformer()), ('tfidf', TfidfTransformer())],
       transformer_weights=None)

In [4]:
train_features = feature_creator.transform(train_data)
test_features = feature_creator.transform(test_data)

In [5]:
with open (file_directory+"preprocessed_train_data.pkl", "wb") as handle:
    pickle.dump(train_data, handle)

with open (file_directory+"preprocessed_test_data.pkl", "wb") as handle:
    pickle.dump(test_data, handle)
    
with open (file_directory+"preprocessed_train_labels.pkl", "wb") as handle:
    pickle.dump(train_y, handle)

with open (file_directory+"preprocessed_test_labels.pkl", "wb") as handle:
    pickle.dump(test_y, handle)